# Creates SBML Models for Sub-SBML
Note: SBML for Bistable Switch already created in Adding rR2 notebook. File name is BiSwitch_CRN.xml

1. Simple rR1 diffusion model (Agrima's atc model)
1. Advanced diffusion of rR1 (Agrima's dna export model)
1. External source with rI# (try integration)

# Bistable Switch
SBML file down below

In [1]:
from biocrnpyler import *
from genelet import *
import pylab as plt
import numpy as np

In [2]:
# Create CRN for bistable switch

### Define Species #####
Core1_off = Species ("Core1")
Core2_off = Species ("Core2")

# Call on genelet code
# Create genelet circuit components with transcript, DNA activator, RNA inhbitor ####
Core1 = Genelet(Core1_off, transcript = "rR1", activator = "dA2", inhibitor = "rR2" )
Core2 = Genelet(Core2_off, transcript = "rR2", activator = "dA1", inhibitor = "rR1" )

# Create Mixture named "Switch_test" that contains Bistable Switch components
# Components are Core 1 and Core 2
# Parameters are in default parameters file

M_BiSwitch = Mixture(name = "Switch_test", components = [Core1, Core2], parameter_file = "default_parameters.txt")
repr(M_BiSwitch)

# Compile the following reactions using... name of mixture.compile_crn()
CRN_BiSwitch = M_BiSwitch.compile_crn()

# Write SBML file
CRN_BiSwitch.write_sbml_file('BiSwitch_CRN.xml')

print(CRN_BiSwitch.pretty_print())

Species (18) = {0. Core1_OFF, 1. complex[Core1_OFF:dna[dA2]], 2. rna[rR1], 3. dna[dA2], 4. rna[rR2], 5. protein[RNAP], 6. protein[RNAseH], 7. complex[rna[rR2]:dna[dA2]], 8. complex[complex[Core1_OFF:dna[dA2]]:protein[RNAP]], 9. complex[Core1_OFF:protein[RNAP]], 10. complex[complex[rna[rR2]:dna[dA2]]:protein[RNAseH]], 11. Core2_OFF, 12. complex[dna[dA1]:Core2_OFF], 13. dna[dA1], 14. complex[rna[rR1]:dna[dA1]], 15. complex[complex[dna[dA1]:Core2_OFF]:protein[RNAP]], 16. complex[Core2_OFF:protein[RNAP]], 17. complex[complex[rna[rR1]:dna[dA1]]:protein[RNAseH]]}
Reactions (18) = [
0. Core1_OFF + dna[dA2] --> complex[Core1_OFF:dna[dA2]]        
        massaction: k_f(Core1_OFF,dna[dA2])=0.00394*Core1_OFF*dna[dA2]
1. complex[Core1_OFF:dna[dA2]] + rna[rR2] --> Core1_OFF + complex[rna[rR2]:dna[dA2]]        
        massaction: k_f(complex[Core1_OFF:dna[dA2]],rna[rR2])=0.0696*complex[Core1_OFF:dna[dA2]]*rna[rR2]
2. dna[dA2] + rna[rR2] --> complex[rna[rR2]:dna[dA2]]        
        massaction: k

# Produce 2N
SBML file below with DFHBI in separate reaction

In [3]:
# Create CRN of Produce 2N
Prod2_off = Species("Prod2")
Broccoli_Aptamer = Species("BrocApt")

Prod2 = Genelet(Prod2_off, transcript= Broccoli_Aptamer , activator= "dAP"  , inhibitor= "rR1P")
M_Prod2 = Mixture(name = "Produce2_test", components = [Prod2], parameter_file = "default_parameters.txt")
repr(M_Prod2)
CRN_Prod2 = M_Prod2.compile_crn()

# Write SBML
CRN_Prod2.write_sbml_file('Prod2_CRN.xml')

print(CRN_Prod2.pretty_print())

Species (11) = {0. Prod2_OFF, 1. complex[dna[dAP]:Prod2_OFF], 2. BrocApt, 3. dna[dAP], 4. rna[rR1P], 5. protein[RNAP], 6. protein[RNAseH], 7. complex[dna[dAP]:rna[rR1P]], 8. complex[complex[dna[dAP]:Prod2_OFF]:protein[RNAP]], 9. complex[Prod2_OFF:protein[RNAP]], 10. complex[complex[dna[dAP]:rna[rR1P]]:protein[RNAseH]]}
Reactions (9) = [
0. Prod2_OFF + dna[dAP] --> complex[dna[dAP]:Prod2_OFF]        
        massaction: k_f(Prod2_OFF,dna[dAP])=0.00394*Prod2_OFF*dna[dAP]
1. complex[dna[dAP]:Prod2_OFF] + rna[rR1P] --> Prod2_OFF + complex[dna[dAP]:rna[rR1P]]        
        massaction: k_f(complex[dna[dAP]:Prod2_OFF],rna[rR1P])=0.0696*complex[dna[dAP]:Prod2_OFF]*rna[rR1P]
2. dna[dAP] + rna[rR1P] --> complex[dna[dAP]:rna[rR1P]]        
        massaction: k_f(dna[dAP],rna[rR1P])=0.0696*dna[dAP]*rna[rR1P]
3. complex[dna[dAP]:Prod2_OFF] + protein[RNAP] <--> complex[complex[dna[dAP]:Prod2_OFF]:protein[RNAP]]        
        massaction: k_f(complex[dna[dAP]:Prod2_OFF],protein[RNAP])=38.85714285

#### Create CRN for DFHBI and Broc Apt Binding

In [4]:
### Species ###
Broccoli_Aptamer = Species("BrocApt")
DFHBI = Species("DFHBI")
Flu1 = ComplexSpecies([Broccoli_Aptamer,DFHBI])
species = [Broccoli_Aptamer, DFHBI, Flu1]

### Create Reaction ###
rxn1 = Reaction([Broccoli_Aptamer, DFHBI], [Flu1], propensity_type = "massaction", k= 9.96e-5)
rxns = [rxn1]

D = ChemicalReactionNetwork(species = species, reactions = rxns)
_ = D.write_sbml_file('DFHBI.xml')
print(D)

Species = BrocApt, DFHBI, complex_BrocApt_DFHBI
Reactions = [
	BrocApt + DFHBI --> complex_BrocApt_DFHBI        massaction: k_f(BrocApt,DFHBI)=9.96e-05*BrocApt*DFHBI
]


# rR1 external reservoir
## Unimportant reservoir... can use if testing out models individually

# Create External Environment
**Species within External Environment**
* rR1
* Broccoli Aptamer
* DFHBI

In [5]:
from subsbml import createBasicSubsystem, SimpleModel
import libsbml

# Creating external_environment for communication model
external_ss, external_environment = createBasicSubsystem('external_environment')

per_second = external_environment.createNewUnitDefinition(uId = 'per_second',uKind = libsbml.UNIT_KIND_SECOND,
                                                uExponent = -1)
substance = external_environment.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, uExponent = 1)

# Species that will exist in external envirnoment

external_environment.createNewSpecies('rna_rR1_e',sComp = 'external_environment', ListOfAmounts = 0, 
                            sConstant = False, sSubstance = 'substance')
external_environment.createNewSpecies('BrocApt_e',sComp = 'external_environment', ListOfAmounts = 0, 
                            sConstant = False, sSubstance = 'substance')
external_environment.createNewSpecies('DFHBI_e',sComp = 'external_environment', ListOfAmounts = 0, 
                            sConstant = False, sSubstance = 'substance')


# Write to SBML file 
_ = external_ss.writeSBML('external_environment.xml')

# rR1 simple membrane model

rR1_internal $\leftrightarrow$ rR1_external

**Note:** not relevant reaction for RNA transport... can use to model DFHBI diffusion

In [6]:
##### Creating a basic membrane model for IPTG transport####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction
import libsbml

rR1_mb_ss = createNewSubsystem()
model = rR1_mb_ss.createNewModel('rR1_membrane','second','mole','substance')

rR1_mb = SimpleModel(model)

per_second = rR1_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = rR1_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
rR1_mb.createNewCompartment('internal','internal',1e-6,'litre',True)
rR1_mb.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################
 
rR1_mb.createNewSpecies( 'rna_rR1_e','external',1e4,False,'substance')
rR1_mb.createNewSpecies( 'rna_rR1','internal',0,False,'substance')

rR1_mb.createNewParameter( 'kb',2e-3,False,'per_second')
rR1_mb.createNewParameter( 'kd',1e-5,False,'per_second')

# model = IPTG_mb_ss.getSBMLDocument().getModel()
r1_sbml = model.createReaction()
r1 = SimpleReaction(r1_sbml)
r1.setId('r1')
r1.setReversible(True)
r1.createNewReactant('rna_rR1_e',False,1)
r1.createNewProduct('rna_rR1', False, 1)
r1.createRate('kb * rna_rR1_e - kd * rna_rR1')

# Write to SBML file 
_ = rR1_mb_ss.writeSBML('simplemembrane_rR1.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully


# DFHBI Simple Diffusion #
DFHBI_internal $\leftrightarrow$ DFHBI_external
* DFHBI (Produce2N)
* DFHBI_B (Bistable Switch)

In [7]:
##### Creating a basic membrane model for DFHBI transport####
#### Produce 2N #####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction
import libsbml

DFHBI_mb_ss = createNewSubsystem()
model1 = DFHBI_mb_ss.createNewModel('DFHBI_membrane','second','mole','substance')

DFHBI_mb = SimpleModel(model1)

per_second = DFHBI_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = DFHBI_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
DFHBI_mb.createNewCompartment('internal','internal',1e-6,'litre',True)
DFHBI_mb.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

# Create new species for internal and external of membrane
DFHBI_mb.createNewSpecies( 'DFHBI_e','external',1e4,False,'substance')
DFHBI_mb.createNewSpecies( 'DFHBI','internal',0,False,'substance')

DFHBI_mb.createNewParameter( 'kb_D',2e-3,False,'per_second')
DFHBI_mb.createNewParameter( 'kd_D',1e-5,False,'per_second')

# model1 = DFHBI_mb_ss.getSBMLDocument().getModel()

# Create reactions for DFHBI diffusion
r1_sbml = model1.createReaction()
r1 = SimpleReaction(r1_sbml)
r1.setId('r1')
r1.setReversible(True)
r1.createNewReactant('DFHBI_e',False,1)
r1.createNewProduct('DFHBI', False, 1)
r1.createRate('kb_D * DFHBI_e - kd_D * DFBHI')

r2_sbml = model1.createReaction()
r2 = SimpleReaction(r2_sbml)
r2.setId('r2')
r2.setReversible(True)
r2.createNewReactant('DFHBI_e',False,1)
r2.createNewProduct('DFHBI', False, 1)
r2.createRate('kd_D * DFHBI_e - kb_D * DFBHI')

# Write to SBML file 
_ = DFHBI_mb_ss.writeSBML('simple_DFHBI_Prod.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully


In [8]:
##### Creating a basic membrane model for DFHBI transport####
#### Bistable Switch #####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction
import libsbml

DFHBI_mb_ss2 = createNewSubsystem()
model2 = DFHBI_mb_ss2.createNewModel('DFHBI_membrane2','second','mole','substance')

DFHBI_mb2 = SimpleModel(model2)

per_second = DFHBI_mb2.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = DFHBI_mb2.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
DFHBI_mb2.createNewCompartment('internal','internal',1e-6,'litre',True)
DFHBI_mb2.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

# Create internal and external species of membrane model
DFHBI_mb2.createNewSpecies( 'DFHBI_e','external',1e4,False,'substance')
DFHBI_mb2.createNewSpecies( 'DFHBI_B','internal',0,False,'substance')

# Rate parameters for diffusion
DFHBI_mb2.createNewParameter( 'kb_B',2e-3,False,'per_second')
DFHBI_mb2.createNewParameter( 'kd_B',1e-5,False,'per_second')

# model1 = DFHBI_mb_ss.getSBMLDocument().getModel()

# Create diffusion reaction for DFHBI (external <----> internal)
r3_sbml = model2.createReaction()
r3 = SimpleReaction(r3_sbml)
r3.setId('r3')
r3.setReversible(True)
r3.createNewReactant('DFHBI_e',False,1)
r3.createNewProduct('DFHBI_B', False, 1)
r3.createRate('kb_B * DFHBI_e - kd_B * DFBHI_B')

r4_sbml = model2.createReaction()
r4 = SimpleReaction(r4_sbml)
r4.setId('r4')
r4.setReversible(True)
r4.createNewReactant('DFHBI_e',False,1)
r4.createNewProduct('DFHBI_B', False, 1)
r4.createRate('kd_B * DFHBI_e - kb_B * DFBHI_B')

# Write to SBML file 
_ = DFHBI_mb_ss2.writeSBML('simple_DFHBI_Bistable.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully


# rR1 advanced membrane model (SID)
## Use bottom 3 for dsRNA diffusion ##
## From Agrima D's and Ayush P's model of transport systems ##
**See sub-SBML repo for more info**

In [9]:
#### Inspired by Agrima's dnaexp_membrane3.xml ####
#### membrane for Bistable Switch #####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction
import libsbml

# Create complex membrane with transport protein #
rnaexp_mb_ss = createNewSubsystem()

model = rnaexp_mb_ss.createNewModel('rnaexp_membrane','second','mole','substance')

rnaexp_mb = SimpleModel(model)

per_second = rnaexp_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = rnaexp_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
rnaexp_mb.createNewCompartment('internal','internal',1e-6,'litre',True)
rnaexp_mb.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

# Create new species for internal and external RNA_rR1

rnaexp_mb.createNewSpecies( 'rna_rR1_e','external',0,False,'substance')
rnaexp_mb.createNewSpecies( 'rna_rR1','internal',0,False,'substance')

# sid is the transport protein
rnaexp_mb.createNewSpecies( 'sid','internal',1,False,'substance')

# sid and rna_rR1 binding to each other
rnaexp_mb.createNewSpecies( 'rna_rR1_sid','internal',0,False,'substance')

# rate parameters for binding and unbinding of rna from sid
rnaexp_mb.createNewParameter( 'kb',0.2,False,'per_second') #0.2
rnaexp_mb.createNewParameter( 'kd',0.01,False,'per_second') #0.01

model = rnaexp_mb_ss.getSBMLDocument().getModel()

# Create diffusion reaction for rna_rR1
# Diffusion from internal to external enviornemnt (also reversible)
R1 = SimpleReaction(model.createReaction())
R1.setId('R1')
R1.setReversible(True)
R1.createNewReactant('sid',False,1)
R1.createNewReactant('rna_rR1', False, 1)
R1.createNewProduct('rna_rR1_sid', False, 1)
R1.createRate('kb * sid * rna_rR1 - kd * rna_rR1_sid')

R2 = SimpleReaction(model.createReaction())
R2.setId('R2')
R2.setReversible(True)
R2.createNewReactant('rna_rR1_sid',False,1)
R2.createNewProduct('rna_rR1_e',False,1)
R2.createNewProduct('sid',False,1)
R2.createRate('kd * rna_rR1_sid - kb * sid * rna_rR1_e')


# Write to SBML file 
_ = rnaexp_mb_ss.writeSBML('rR1_membrane1_detailed.xml')
print('Model written to a SBML file successfully')


Model written to a SBML file successfully


In [10]:
#### Agrima's dnaexp_membrane3.xml ####
#### membrane for Produce2N #####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction
import libsbml

rnaexp_mb_ss3 = createNewSubsystem()


model3 = rnaexp_mb_ss3.createNewModel('rnaexp_membrane3','second','mole','substance')

rnaexp_mb3 = SimpleModel(model3)

per_second = rnaexp_mb3.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = rnaexp_mb3.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
rnaexp_mb3.createNewCompartment('internal','internal',1e-6,'litre',True)
rnaexp_mb3.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

# Create new species internal and external of membrane
# Note that sid needs to be named differently for simulation to work properly
# Anything with a similar named will be combined despite different compartments

# Sid species
rnaexp_mb3.createNewSpecies( 'sidP','internal',1,False,'substance')
rnaexp_mb3.createNewSpecies( 'sidB','internal',1,False,'substance')

# RNA species
rnaexp_mb3.createNewSpecies( 'rna_rR1_e','external',0,False,'substance')
rnaexp_mb3.createNewSpecies( 'rna_rR1P','internal',0,False,'substance')
rnaexp_mb3.createNewSpecies( 'BrocApt_e','external',0,False,'substance')
rnaexp_mb3.createNewSpecies( 'BrocApt','internal',0,False,'substance')

# RNA binding to SID protein
rnaexp_mb3.createNewSpecies( 'rna_rR1_e_sidP','internal',0,False,'substance')
rnaexp_mb3.createNewSpecies( 'BrocApt_sidB','internal',0,False,'substance')

# Rate parameters
rnaexp_mb3.createNewParameter( 'kb1',0.01,False,'per_second') #0.2
rnaexp_mb3.createNewParameter( 'kd1',0.2,False,'per_second') #0.01
rnaexp_mb3.createNewParameter( 'kc',0.2,False,'per_second') #0.2
rnaexp_mb3.createNewParameter( 'ke',0.01,False,'per_second') #0.01

model3 = rnaexp_mb_ss3.getSBMLDocument().getModel()

# Create diffusion reaction for rna_rR1 (external <--> internal)
R3 = SimpleReaction(model3.createReaction())
R3.setId('R3')
R3.setReversible(True)
R3.createNewReactant('sidP',False,1)
R3.createNewReactant('rna_rR1_e', False, 1)
R3.createNewProduct('rna_rR1_e_sidP', False, 1)
R3.createRate('kb1 * sidP * rna_rR1_e - kd1 * rna_rR1_e_sidP')

R4 = SimpleReaction(model3.createReaction())
R4.setId('R4')
R4.setReversible(True)
R4.createNewReactant('rna_rR1_e_sidP',False,1)
R4.createNewProduct('rna_rR1P',False,1)
R4.createNewProduct('sidP',False,1)
R4.createRate('kd1 * rna_rR1_e_sidP - kb1 * sidP * rna_rR1P')



#### Broccoli Apt ####
# Create diffusion reaction for broccoli aptamer (external <--> internal)
R5 = SimpleReaction(model3.createReaction())
R5.setId('R5')
R5.setReversible(True)
R5.createNewReactant('sidB',False,1)
R5.createNewReactant('BrocApt', False, 1)
R5.createNewProduct('BrocApt_sidB', False, 1)
R5.createRate('kc * sidB * BrocApt - ke * BrocApt_sidB')

R6 = SimpleReaction(model3.createReaction())
R6.setId('R6')
R6.setReversible(True)
R6.createNewReactant('BrocApt_sidB',False,1)
R6.createNewProduct('BrocApt_e',False,1)
R6.createNewProduct('sidB',False,1)
R6.createRate('ke * BrocApt_sidB - kc * sidB * BrocApt_e')


# Write to SBML file 
_ = rnaexp_mb_ss3.writeSBML('rR1_P_membrane1_detailed.xml')
print('Model written to a SBML file successfully')


Model written to a SBML file successfully


In [11]:
#### Agrima's dnaexp_membrane3.xml ####
#### membrane for Produce2N #####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction
import libsbml

rnaexp_mb_ss2 = createNewSubsystem()


model2 = rnaexp_mb_ss2.createNewModel('rnaexp_membrane2','second','mole','substance')

rnaexp_mb2 = SimpleModel(model2)

per_second = rnaexp_mb2.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = rnaexp_mb2.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
rnaexp_mb2.createNewCompartment('internal','internal',1e-6,'litre',True)
rnaexp_mb2.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################
rnaexp_mb2.createNewSpecies( 'sidB','internal',1,False,'substance')
rnaexp_mb2.createNewSpecies( 'BrocApt_e','external',0,False,'substance')
rnaexp_mb2.createNewSpecies( 'BrocApt','internal',0,False,'substance')
rnaexp_mb2.createNewSpecies( 'BrocApt_sidB','internal',0,False,'substance')

rnaexp_mb2.createNewParameter( 'kc',0.2,False,'per_second') #0.2
rnaexp_mb2.createNewParameter( 'ke',0.01,False,'per_second') #0.01

model2 = rnaexp_mb_ss2.getSBMLDocument().getModel()

##### Broccoli Apt ####
R5 = SimpleReaction(model2.createReaction())
R5.setId('R5')
R5.setReversible(True)
R5.createNewReactant('sidB',False,1)
R5.createNewReactant('BrocApt', False, 1)
R5.createNewProduct('BrocApt_sidB', False, 1)
R5.createRate('kc * sidB * BrocApt - ke * BrocApt_sidB')

R6 = SimpleReaction(model2.createReaction())
R6.setId('R6')
R6.setReversible(True)
R6.createNewReactant('BrocApt_sidB',False,1)
R6.createNewProduct('BrocApt_e',False,1)
R6.createNewProduct('sidB',False,1)
R6.createRate('ke * BrocApt_sidB - kc * sidB * BrocApt_e')


# Write to SBML file 
_ = rnaexp_mb_ss2.writeSBML('rR1_Broc_membrane1_detailed.xml')
print('Model written to a SBML file successfully')


Model written to a SBML file successfully
